# Find cluster related terms using LIME

## Load the data

In [1]:
root_dir = '../..'
data_dir = 'data'
corpus_dir = 'corpus'
src_dir = 'src'

In [2]:
import os 
import sys

In [3]:
sys.path.append(os.path.join(root_dir, src_dir))

In [4]:
dataset_name = 'alaska'
test_name = 'sampling_records_test2'

In [5]:
corpus_filename = f'{dataset_name}_corpus.json'
corpus_filepath = os.path.join(root_dir, data_dir, corpus_dir, corpus_filename)

In [6]:
chunks_filename = f'{dataset_name}_chunks.json'
chunks_filepath = os.path.join(root_dir, data_dir, corpus_dir, chunks_filename)

In [7]:
from training import TrainingCorpus

In [8]:
corpus = TrainingCorpus()
corpus.load(corpus_filepath)
corpus.load_chunks(chunks_filepath)

---

## Load the classification model

Check if GPU is available

In [9]:
import torch
print(torch.cuda.is_available())

True


In [10]:
from model import BertModel

Using TensorFlow backend.


In [11]:
model_dir = f'models/{dataset_name}_bert'
model_dir_path = os.path.join(root_dir, data_dir, model_dir)

In [12]:
model = BertModel(model_dir_path, batch_size=512, use_cuda=True, from_tf=True)

All TF 2.0 model weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


---

## Sample data

### Load the embedding model

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained(model_dir_path)
embedding_model = AutoModel.from_pretrained(model_dir_path, from_tf=True)
_ = embedding_model.to(device)

All TF 2.0 model weights were used when initializing DistilBertModel.

All the weights of DistilBertModel were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertModel for predictions without further training.


### Get embeddings

In [15]:
def get_batch(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i: i + batch_size]

In [16]:
import numpy as np

In [17]:
texts = [' '.join(corpus.get_tokens(doc_id)) for doc_id in corpus.docs]
embeddings = []
batch_size = 512

for batch in get_batch(texts, batch_size):
    encoded_batch = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')
    encoded_batch.to(device)
    
    with torch.no_grad():
        batch_embeddings = embedding_model(**encoded_batch)['last_hidden_state'][:,0]
        batch_embeddings = batch_embeddings.cpu().numpy()
        embeddings.append(batch_embeddings)

# concatenate all batch embeddings
embeddings = np.concatenate(embeddings, axis=0)
embeddings.shape

(2171, 768)

### Free memory

In [18]:
del(embedding_model)
torch.cuda.empty_cache()

### Sample records using K-means

In [19]:
label_to_idx_map = model.label_to_data_idx(corpus)

In [20]:
from sample import KMeansSampler

sampler = KMeansSampler(corpus, embeddings, label_to_idx_map, min_size=25, max_size=50)
label_to_sample_idx_map = {l: sampler.sample_data(l) for l in label_to_idx_map}

## Find relevant terms for each cluster label using LIME

### Instantiate TermFinder

In [21]:
from termfinder import LimeTermFinder

In [22]:
term_finder = LimeTermFinder(model, corpus)

### Retrieve relevant terms using LimeTermFinder

In [23]:
from tqdm.notebook import tqdm

In [24]:
df_data = []

In [25]:
import time
import datetime

start_time = time.time()

for label_idx, data_idxs in tqdm(label_to_sample_idx_map.items()):
    for data_idx in tqdm(data_idxs, desc=f'Relevant terms for entity {label_idx}', leave=False):
        relevant_terms = term_finder.get_relevant_terms(data_idx, label_idx)

        if relevant_terms:
            for term, weight in relevant_terms.items():
                dict_entry = {'label': corpus.labels[label_idx], 
                              'term': term, 
                              'weight': weight, 
                              'data_id': corpus.docs[data_idx]}
                df_data.append(dict_entry)

end_time = time.time()

In [27]:
delta_t = end_time - start_time
elapsed_time = str(datetime.timedelta(seconds=delta_t))
print(f'Elapsed time: {elapsed_time}')

Elapsed time: 0:47:20.545807


Build a DataFrame out of `df_data`

In [28]:
df_data[:3]

[{'label': 'ENTITY#44',
  'term': 'd32oo',
  'weight': 0.849248541668861,
  'data_id': 34},
 {'label': 'ENTITY#44',
  'term': 'd3200',
  'weight': 0.8515586032016936,
  'data_id': 116},
 {'label': 'ENTITY#44',
  'term': 'd3200_24_2mp_digital_slr_camera_w_af',
  'weight': 0.9182223430445601,
  'data_id': 32}]

In [29]:
import pandas as pd

In [30]:
relevant_terms_df = pd.DataFrame(df_data)

In [31]:
relevant_terms_df.head()

,label,term,weight,data_id
0,ENTITY#44,d32oo,0.849249,34
1,ENTITY#44,d3200,0.851559,116
2,ENTITY#44,d3200_24_2mp_digital_slr_camera_w_af,0.918222,32
3,ENTITY#44,nikon_d3200,0.589502,56
4,ENTITY#44,vr,0.045263,56


---
## Save retrieved terms to a file

In [32]:
terms_dir = 'terms'
filename = f'relevant_terms_{dataset_name}_bert_{test_name}.csv'
filepath = os.path.join(root_dir, data_dir, terms_dir, filename)
filepath

'../../data/terms/relevant_terms_alaska_bert_sampling_records_test2.csv'

In [33]:
relevant_terms_df.to_csv(filepath, encoding='utf-8', index=False)

---